In [ ]:
from openfermion.hamiltonians import fermi_hubbard
from openfermion.ops import QubitOperator, FermionOperator
#from openfermion.linalg import eigenspectrum
import sys
from os import cpu_count
from time import time
sys.path.append("../")
from mappings import hubbard_mapping_helper as mapper
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
import numpy as np
from scipy.optimize import minimize

from math import pi
from copy import deepcopy
from simulator.logical_vector_simulator import logical_state_simulation#simulate_parallel, prepare_stabilizers
from simulator.logical_vector_construction import reduce_to_logical_plus
from simulator.noise import single_qubit_depolarizing, parallel_single_qubit_depolarizing, noiseless
from simulator.utils import expectation_value, check_mapping, anticom
import pickle
from random import shuffle
from itertools import combinations, product 

#### This notebook is used to generate the data for VQE numerical experiments

In [ ]:
mappings = ['jw','lw','gse','aqm','gse3']
vqe_mappings = ['noiseless'] + mappings
mapping_shots = {
    'jw':1000,
    'lw':1000,
    'gse':1000,
    'aqm':1000,
    'gse3':1000,
}

target_shots = 100

x_dim = 2
y_dim = 3
t = 1.0
U = 0.5
mu = -0.7
h = 0.0
fer_ham = fermi_hubbard(x_dimension = x_dim,
                  y_dimension = y_dim,
                  tunneling = t,
                  coulomb = U,
                  chemical_potential=mu,
                  magnetic_field=h,
                  periodic=False,
                  spinless=False,
                  particle_hole_symmetry=False)

E_maximally_mixed = 4.949999999999999
E0 = -3.7074231095566774

layers = 1
num_angles = 32*layers


stat_noise_rate = 0#0.0001
gate_noise_rate = 0.01
if stat_noise_rate==0.0:
    stat_noise_model = noiseless()
else:
    stat_noise_model = single_qubit_depolarizing(stat_noise_rate)
if gate_noise_rate==0.0:
    gate_noise_model = noiseless()
else:
    #gate_noise_model = single_qubit_depolarizing(gate_noise_rate)
    gate_noise_model = parallel_single_qubit_depolarizing(gate_noise_rate)
opt_iters = 10
anglewise=True
l_rate = 0.25

num_shots = 300 
if stat_noise_rate ==0 and gate_noise_rate==0:
    num_shots = 1

cpus = 6#cpu_count()-1

In [ ]:
mapping_hams = {}
mapping_stabs = {}
mapping_qubits = {
    'jw':12,
    'lw':14,
    'gse':16,
    'aqm':25,
    'gse3':42}
mapping_fixed_ops = {}
mapping_energies = {'jw':[],'lw':[],'gse':[],'aqm':[],'gse3':[]}
mapping_circuit = {
    'jw':mapper.jw_circuit,
    'lw':mapper.lw_circuit,
    'gse':mapper.gse_circuit,
    'aqm':mapper.aqm_circuit,
    'gse3':mapper.gse_dist3_circuit}
mapping_Aops = {}
mapping_Bops = {}
mapping_mappers = {
    'jw':mapper.jw_mapping,
    'lw':mapper.lw_mapping,
    'gse':mapper.gse_mapping,
    'aqm':mapper.aqm_mapping,
    'gse3':mapper.gse_dist3_mapping}
for mapping in mappings:
    mapping_hams[mapping], mapping_stabs[mapping], mapping_Aops[mapping], mapping_Bops[mapping] = mapping_mappers[mapping](fer_ham)
mapping_stabs['jw'] = []

In [ ]:
print(type(mapping_hams['jw']))

In [ ]:
avg_Aop_weights = {}
avg_Bop_weights = {}
avg_stab_weights = {}

def op_weight(op):
    return len(list(op.terms.items())[0][0])

for mapping in mappings:
    Aop_list = list(mapping_Aops[mapping].items())
    summed_weight = 0
    for j in range(len(Aop_list)):
        summed_weight += op_weight(Aop_list[j][1])
    avg_Aop_weights[mapping] = summed_weight/len(Aop_list)

    Bop_list = list(mapping_Bops[mapping].items())
    summed_weight = 0
    for j in range(len(Bop_list)):
        summed_weight += op_weight(Bop_list[j][1])
    avg_Bop_weights[mapping] = summed_weight/len(Bop_list)

    if len(mapping_stabs[mapping])!=0:
        summed_weight = 0
        for j in range(len(mapping_stabs[mapping])):
            summed_weight += op_weight(mapping_stabs[mapping][j])
        avg_stab_weights[mapping] = summed_weight/len(mapping_stabs[mapping])
    else:
        avg_stab_weights[mapping] = 0

print(avg_Aop_weights)
print(avg_Bop_weights)
print(avg_stab_weights)

In [ ]:
mapping_fixed_ops['lw'] = [ 
              (-1)*mapping_Bops['lw'][0],mapping_Bops['lw'][1],
              (-1)*mapping_Bops['lw'][2],mapping_Bops['lw'][3],
              mapping_Bops['lw'][4],#mapping_Bops['lw'][5],
              (-1)*mapping_Bops['lw'][6],mapping_Bops['lw'][7],
              (-1)*mapping_Bops['lw'][8],mapping_Bops['lw'][9],
              mapping_Bops['lw'][10],#mapping_Bops['lw'][11]
              ]
mapping_fixed_ops['gse'] = [
              (-1)*mapping_Bops['gse'][0],mapping_Bops['gse'][1],
              (-1)*mapping_Bops['gse'][2],mapping_Bops['gse'][3],
              mapping_Bops['gse'][4],#mapping_Bops['gse'][5],
              (-1)*mapping_Bops['gse'][6],mapping_Bops['gse'][7],
              (-1)*mapping_Bops['gse'][8],mapping_Bops['gse'][9],
              mapping_Bops['gse'][10],#mapping_Bops['gse'][11]
              ]
mapping_fixed_ops['jw'] = [
              (-1)*mapping_Bops['jw'][0],mapping_Bops['jw'][1],
              (-1)*mapping_Bops['jw'][2],mapping_Bops['jw'][3],
              mapping_Bops['jw'][4],#mapping_Bops['jw'][5],
              (-1)*mapping_Bops['jw'][6],mapping_Bops['jw'][7],
              (-1)*mapping_Bops['jw'][8],mapping_Bops['jw'][9],
              mapping_Bops['jw'][10],#mapping_Bops['jw'][11] #
              ]
mapping_fixed_ops['gse3'] = [
              (-1)*mapping_Bops['gse3'][0],mapping_Bops['gse3'][1],
              (-1)*mapping_Bops['gse3'][2],mapping_Bops['gse3'][3],
              mapping_Bops['gse3'][4],#mapping_Bops['gse3'][5],
              #mapping_Bops['gse3'][6],
              (-1)*mapping_Bops['gse3'][7],mapping_Bops['gse3'][8],
              (-1)*mapping_Bops['gse3'][9],mapping_Bops['gse3'][10],
              mapping_Bops['gse3'][11],#mapping_Bops['gse3'][12],
              #mapping_Bops['gse3'][13]
              ]
mapping_stabs['gse3'].append(mapping_Bops['gse3'][6])
mapping_stabs['gse3'].append(mapping_Bops['gse3'][13])
mapping_fixed_ops['aqm'] = [
              (-1)*mapping_Bops['aqm'][0],mapping_Bops['aqm'][1],
              (-1)*mapping_Bops['aqm'][2],mapping_Bops['aqm'][3],
              mapping_Bops['aqm'][4],#mapping_Bops['aqm'][5],
              (-1)*mapping_Bops['aqm'][6],mapping_Bops['aqm'][7],
              (-1)*mapping_Bops['aqm'][8],mapping_Bops['aqm'][9],
              mapping_Bops['aqm'][10],#mapping_Bops['aqm'][11]
              ]

mapping_all_angles = {}
mapping_updated_angles = {}
mapping_energies = {}

#for mapping in mappings:
#    print(mapping)
#    check_mapping(mapping_Aops[mapping],mapping_Bops[mapping], mapping_stabs[mapping])

In [ ]:
init_angles = [np.random.rand() for j in range(num_angles)]
ferm_circuit = mapper.get_ferm_circuit(init_angles)

lw_schedule = mapper.lw_circuit(ferm_circuit)
gse_schedule = mapper.gse_circuit(ferm_circuit)
jw_schedule = mapper.jw_circuit(ferm_circuit)
gse_dist3_schedule = mapper.gse_dist3_circuit(ferm_circuit)

if stat_noise_rate==0.0:
    stat_noise_model = noiseless()
else:
    stat_noise_model = single_qubit_depolarizing(stat_noise_rate)
if gate_noise_rate==0.0:
    gate_noise_model = noiseless()
else:
    gate_noise_model = single_qubit_depolarizing(gate_noise_rate)

In [ ]:
mapping_syndromes = {}
for mapping in ['jw','lw','gse','aqm','gse3']:
    mapping_syndromes[mapping] = []

In [ ]:
# Define the cost function
def energy_cost_alt(
    fer_ham, 
    phys_qubits, 
    schedule, 
    stabs, 
    init_fixed_ops, 
    stat_noise_model, 
    gate_noise_model, 
    shots,
    syndromes=[],
    get_trace=False,
    cpus=1):
    """
    """
    if len(syndromes)==0:
        blocks_type='code'
    else:
        blocks_type='custom'
    num_qubits = phys_qubits
    rounds = shots
    block_entries = 0
    attempts = 0
    while block_entries==0:
        attempts +=1
        (state, block_entries, ideal_state, depth, data)  = logical_state_simulation(
            stabilizers = stabs,
            n_phys_qubits = num_qubits,
            rounds = rounds,
            stat_noise = stat_noise_model,
            gate_noise = gate_noise_model,
            phys_circuit = schedule,
            logical_operators = init_fixed_ops,
            d_matrix_blocks = blocks_type,
            block_numbers = syndromes,
            num_processes=cpus
            )
        if block_entries==0:
            print('------Zero block entries------')

    if blocks_type=='custom':
        codespace_state = state[tuple(np.asarray([0 for j in range(len(stabs))]))]
        codespace_trace = np.real(np.trace(codespace_state))
    else:
        codespace_state = state
        codespace_trace = np.real(np.trace(codespace_state))
    
    updated_stabs = data[0]
    qubit_order = data[1]
    fixed_positions = data[2]
    fixed_ops = data[3]
    other_ops = data[4]

    energy = np.real(expectation_value(
        fer_ham,
        state,
        updated_stabs,
        qubit_order,
        fixed_positions, 
        fixed_ops, 
        other_ops
        ))

    if get_trace:
        return energy, codespace_trace*block_entries/shots
    else:
        return float(energy)

In [ ]:
def gradient_component(
    angles,
    #layers,
    target_angle,
    mapping,
    stat_noise_model=noiseless(),
    gate_noise_model=noiseless(),
    shots=1,
    syndromes=[],
    cpus=1
    ):
    """
    Get the angle increment via the parameter shift rule
    """ 
    if target_angle % 32 < 14: # <-- Hopping gate, use 4 shift rule from https://arxiv.org/abs/2107.12390
        # weighting constants
        d1 = (np.sqrt(2)+1)/(2*np.sqrt(2))
        d2 = (np.sqrt(2)-1)/(2*np.sqrt(2))
        # angle shift constants
        alpha = pi/4
        beta = 3*pi/4
        # create 4 evaluation schedules
        shift_1_angles = deepcopy(angles)
        shift_1_angles[target_angle] += alpha
        shift_1_ferm_circuit = mapper.get_ferm_circuit(shift_1_angles)
        shift_1_schedule = mapping_circuit[mapping](shift_1_ferm_circuit)
        shift_1_energy = energy_cost_alt(
            mapping_hams[mapping], 
            mapping_qubits[mapping], 
            shift_1_schedule, 
            mapping_stabs[mapping], 
            mapping_fixed_ops[mapping], 
            stat_noise_model, 
            gate_noise_model,
            shots,
            syndromes,
            cpus=cpus)
        shift_2_angles = deepcopy(angles)
        shift_2_angles[target_angle] -= alpha
        shift_2_ferm_circuit = mapper.get_ferm_circuit(shift_2_angles)
        shift_2_schedule = mapping_circuit[mapping](shift_2_ferm_circuit)
        shift_2_energy = energy_cost_alt(
            mapping_hams[mapping], 
            mapping_qubits[mapping], 
            shift_2_schedule, 
            mapping_stabs[mapping], 
            mapping_fixed_ops[mapping], 
            stat_noise_model, 
            gate_noise_model,
            shots,
            syndromes,
            cpus=cpus)
        shift_3_angles = deepcopy(angles)
        shift_3_angles[target_angle] += beta
        shift_3_ferm_circuit = mapper.get_ferm_circuit(shift_3_angles)
        shift_3_schedule = mapping_circuit[mapping](shift_3_ferm_circuit)
        shift_3_energy = energy_cost_alt(
            mapping_hams[mapping], 
            mapping_qubits[mapping], 
            shift_3_schedule, 
            mapping_stabs[mapping], 
            mapping_fixed_ops[mapping], 
            stat_noise_model, 
            gate_noise_model,
            shots,
            syndromes,
            cpus=cpus)
        shift_4_angles = deepcopy(angles)
        shift_4_angles[target_angle] -= beta
        shift_4_ferm_circuit = mapper.get_ferm_circuit(shift_4_angles)
        shift_4_schedule = mapping_circuit[mapping](shift_4_ferm_circuit)
        shift_4_energy = energy_cost_alt(
            mapping_hams[mapping], 
            mapping_qubits[mapping], 
            shift_4_schedule, 
            mapping_stabs[mapping], 
            mapping_fixed_ops[mapping], 
            stat_noise_model, 
            gate_noise_model,
            shots,
            syndromes,
            cpus=cpus)
        gradient_component = np.real(d1*(shift_1_energy - shift_2_energy) - d2*(shift_3_energy - shift_4_energy))
    else: # <-- Number gate, use 2 shift rule
        shift_up = deepcopy(angles)
        shift_up[target_angle] += pi/4
        shifted_up_ferm_circuit = mapper.get_ferm_circuit(shift_up)
        shifted_up_schedule = mapping_circuit[mapping](shifted_up_ferm_circuit)
        shifted_up_energy = energy_cost_alt(
            mapping_hams[mapping], 
            mapping_qubits[mapping], 
            shifted_up_schedule, 
            mapping_stabs[mapping], 
            mapping_fixed_ops[mapping], 
            stat_noise_model, 
            gate_noise_model,
            shots,
            syndromes,
            cpus=cpus)
        shift_down = deepcopy(angles)
        shift_down[target_angle] -= pi/4
        shifted_down_ferm_circuit = mapper.get_ferm_circuit(shift_down)
        shifted_down_schedule = mapping_circuit[mapping](shifted_down_ferm_circuit)
        shifted_down_energy = energy_cost_alt(
            mapping_hams[mapping], 
            mapping_qubits[mapping], 
            shifted_down_schedule, 
            mapping_stabs[mapping], 
            mapping_fixed_ops[mapping], 
            stat_noise_model, 
            gate_noise_model,
            shots,
            syndromes)
        gradient_component = np.real((shifted_up_energy - shifted_down_energy))
    return gradient_component

In [ ]:
def VQE(
    #layers,
    mapping,
    l_rate,
    opt_iters,
    stat_noise_model=noiseless(),
    gate_noise_model=noiseless(),
    shots=1,
    syndromes=[],
    init_angles=None,
    anglewise_opt=True,
    randomize_opt=False,
    monitor_energy=True, 
    cpus=1
    ):
    """
    """
    if init_angles is None:
        angles = np.random.rand(32*layers)
    else:
        angles=init_angles
    all_angles = [angles]
    energies=[]
    ferm_circuit = mapper.get_ferm_circuit(angles)
    schedule = mapping_circuit[mapping](ferm_circuit)
    start = time()
    energies.append(energy_cost_alt(
        mapping_hams[mapping], 
        mapping_qubits[mapping], 
        schedule, 
        mapping_stabs[mapping], 
        mapping_fixed_ops[mapping], 
        stat_noise_model, 
        gate_noise_model, 
        shots,
        syndromes,
        get_trace=False,
        cpus=cpus))
    end = time()
    print(f'Begin: current energy = {energies[-1]}')
    if monitor_energy:
        print(f'Expected time to finish {(3*opt_iters*len(angles)*(end-start))/60} minutes'  )
    else:
        print(f'Expected time to finish {(2*opt_iters*len(angles)*(end-start))/60} minutes'  )

    if anglewise_opt:
        for j in range(opt_iters):
            if randomize_opt:
                angle_order = [k for k in range(len(angles))]
                shuffle(angle_order)
            else:
                angle_order = [k for k in range(len(angles))]
            for k in angle_order:
                angles[k] -= l_rate*gradient_component(
                    angles,
                    #layers,
                    k,
                    mapping,
                    stat_noise_model,
                    gate_noise_model,
                    shots,
                    syndromes,
                    cpus)
                # Evaluate energy to track progress
                if monitor_energy:
                    #ferm_circuit = mapper.get_ferm_circuit(angles,layers)
                    ferm_circuit = mapper.get_ferm_circuit(angles)
                    schedule = mapping_circuit[mapping](ferm_circuit)
                    energies.append(energy_cost_alt(
                        mapping_hams[mapping], 
                        mapping_qubits[mapping], 
                        schedule, 
                        mapping_stabs[mapping], 
                        mapping_fixed_ops[mapping], 
                        stat_noise_model, 
                        gate_noise_model, 
                        shots,
                        syndromes,
                        get_trace=False,
                        cpus=cpus))
                    print(f'Iter {j+1} of {opt_iters}, Angle {k+1} of {len(angles)}: current energy = {energies[-1]}')
                mapping_all_angles[mapping].append(angles)
    else:
        for j in range(opt_iters):
            gradient_vector = np.asarray([0 for j in range(len(angles))])
            for k in range(len(angles)):
                print(f'Iter {j+1} of {opt_iters}, gradient component {k+1} of {len(angles)}')
                gradient_vector[k] = gradient_component(
                    angles,
                    layers,
                    k,
                    mapping,
                    stat_noise_model,
                    gate_noise_model,
                    shots,
                    syndromes,
                    cpus)
            # Update angles
            angles = list(np.asarray(angles) - l_rate*gradient_vector)
            all_angles.append(angles)
            # Evaluate energy to track progress
            if monitor_energy:
                ferm_circuit = mapper.get_ferm_circuit(angles)
                schedule = mapping_circuit[mapping](ferm_circuit)
                energies.append(energy_cost_alt(
                    mapping_hams[mapping], 
                    mapping_qubits[mapping], 
                    schedule, 
                    mapping_stabs[mapping], 
                    mapping_fixed_ops[mapping], 
                    noiseless(), 
                    noiseless(), 
                    1,
                    syndromes,
                    get_trace=False,
                    cpus=cpus))
                print(f'Iter {j+1}: current energy = {energies[-1]}')
    if monitor_energy:
        return all_angles, energies
    else:
        return all_angles

In [ ]:
mapping_labels = {
    'noiseless': r'Noiseless',
    'jw': r'JW [[12,12,1]]', 
    'lw': r'Compact [[14,12,1]]', 
    'gse': r'GSE [[16,10,1]]',
    'aqm': r'GAQM [[25,12,2]]',
    'gse3': r'GSE [[42,10,3]]'
}

mapping_colors = {}
mapping_colors['noiseless'] = 'black'
for j in range(len(mappings)):
    mapping_colors[mappings[j]] = list(plt.rcParams['axes.prop_cycle'])[j]['color']

#### Optional: pull in data from previous run to plot

In [ ]:
#with open(f'noisy_VQE_data_updated_code_9-12.pkl', 'rb') as f:
#   experimental_data = pickle.load(f)
#   mapping_energies=experimental_data['energies']

### VQE otimizations

#### First noiseless

In [ ]:
mapping='noiseless'
mapping_all_angles[mapping].append(init_angles)
mapping_updated_angles['noiseless'], mapping_energies['noiseless'] = VQE(
    'gse',
    l_rate,
    opt_iters,
    init_angles=deepcopy(init_angles),
    anglewise_opt=anglewise,
    cpus=1)

In [ ]:
fig = plt.figure(figsize=(8,6),dpi=200)
plt.plot(range(len(mapping_energies['noiseless'])),mapping_energies['noiseless'],label=mapping_labels['noiseless'],color = mapping_colors['noiseless'])
plt.hlines(E0, 0, len(mapping_energies['noiseless']),label='$E_0$',color='gray',linestyles='dashed')
plt.legend()
plt.xlabel(f'Evaluations')
plt.ylabel(rf'$\langle H \rangle$')

plt.margins(x=0)
plt.show()

#### 12-qubit Jordan-Wigner

In [ ]:
mapping = 'jw'
mapping_all_angles[mapping].append(init_angles)
mapping_updated_angles[mapping], mapping_energies[mapping] = VQE(
    mapping,
    l_rate,
    opt_iters,
    init_angles=deepcopy(init_angles),
    syndromes=mapping_syndromes[mapping],
    stat_noise_model=stat_noise_model,
    gate_noise_model=gate_noise_model,
    shots=mapping_shots[mapping],
    anglewise_opt=anglewise,
    cpus=cpus)

In [ ]:
fig = plt.figure(figsize=(8,6),dpi=200)
plt.plot(range(len(mapping_energies['noiseless'])),mapping_energies['noiseless'],label=mapping_labels['noiseless'],color = mapping_colors['noiseless'])
for mapping in ['jw']:
    plt.plot(range(len(mapping_energies[mapping])),mapping_energies[mapping],label=mapping_labels[mapping],color = mapping_colors[mapping])
plt.hlines(E0, 0, len(mapping_energies['noiseless']),label='$E_0$',color='gray',linestyles='dashed')
plt.legend()
plt.xlabel(f'evaluations',color='black')
plt.ylabel(rf'$\langle H \rangle$',color='black')
plt.tick_params(axis='x', colors='black')
plt.tick_params(axis='y', colors='black')

plt.margins(x=0)
plt.show()

#### 15-qubit Compact encoding

In [ ]:
mapping = 'lw'
mapping_all_angles[mapping].append(init_angles)
mapping_updated_angles[mapping], mapping_energies[mapping] = VQE(
    mapping,
    l_rate,
    opt_iters,
    init_angles=deepcopy(init_angles),
    syndromes=mapping_syndromes[mapping],
    stat_noise_model=stat_noise_model,
    gate_noise_model=gate_noise_model,
    shots=mapping_shots[mapping],
    anglewise_opt=anglewise,
    cpus=cpus)

In [ ]:
fig = plt.figure(figsize=(8,6),dpi=200)
plt.plot(range(len(mapping_energies['noiseless'])),mapping_energies['noiseless'],label=mapping_labels['noiseless'],color = mapping_colors['noiseless'])
for mapping in ['jw','lw']:
    plt.plot(range(len(mapping_energies[mapping])),mapping_energies[mapping],label=mapping_labels[mapping],color = mapping_colors[mapping])
plt.hlines(E0, 0, len(mapping_energies['noiseless']),label='$E_0$',color='gray',linestyles='dashed')
plt.legend()
plt.xlabel(f'evaluations',color='black')
plt.ylabel(rf'$\langle H \rangle$',color='black')
plt.tick_params(axis='x', colors='black')
plt.tick_params(axis='y', colors='black')
plt.margins(x=0)
plt.show()

#### 16-qubit GSE

In [ ]:
mapping = 'gse'
mapping_all_angles[mapping].append(init_angles)
mapping_updated_angles[mapping], mapping_energies[mapping] = VQE(
    mapping,
    l_rate,
    opt_iters,
    init_angles=deepcopy(init_angles),
    syndromes=mapping_syndromes[mapping],
    stat_noise_model=stat_noise_model,
    gate_noise_model=gate_noise_model,
    shots=mapping_shots[mapping],
    anglewise_opt=anglewise,
    cpus=cpus)

In [ ]:
fig = plt.figure(figsize=(8,6),dpi=200)
plt.plot(range(len(mapping_energies['noiseless'])),mapping_energies['noiseless'],label=mapping_labels['noiseless'],color = mapping_colors['noiseless'])
for mapping in ['jw','lw','gse']:
    plt.plot(range(len(mapping_energies[mapping])),mapping_energies[mapping],label=mapping_labels[mapping],color = mapping_colors[mapping])
plt.hlines(E0, 0, len(mapping_energies['noiseless']),label='$E_0$',color='gray',linestyles='dashed')
plt.legend()
plt.xlabel(f'evaluations',color='black')
plt.ylabel(rf'$\langle H \rangle$',color='black')
plt.tick_params(axis='x', colors='black')
plt.tick_params(axis='y', colors='black')

plt.margins(x=0)
plt.show()

#### 25-qubit AQM

In [ ]:
mapping = 'aqm'
mapping_all_angles[mapping].append(init_angles)
mapping_updated_angles[mapping], mapping_energies[mapping] = VQE(
    mapping,
    l_rate,
    opt_iters,
    init_angles=deepcopy(init_angles),
    syndromes=mapping_syndromes[mapping],
    stat_noise_model=stat_noise_model,
    gate_noise_model=gate_noise_model,
    shots=mapping_shots[mapping],
    anglewise_opt=anglewise,
    cpus=cpus)

In [ ]:
fig = plt.figure(figsize=(8,6),dpi=200)
plt.plot(range(len(mapping_energies['noiseless'])),mapping_energies['noiseless'],label=mapping_labels['noiseless'],color = mapping_colors['noiseless'])
for mapping in ['jw','lw','gse','aqm']:
    plt.plot(range(len(mapping_energies[mapping])),mapping_energies[mapping],label=mapping_labels[mapping],color = mapping_colors[mapping])
plt.hlines(E0, 0, len(mapping_energies['noiseless']),label='$E_0$',color='gray',linestyles='dashed')
plt.legend()
plt.xlabel(f'evaluations',color='black')
plt.ylabel(rf'$\langle H \rangle$',color='black')
plt.tick_params(axis='x', colors='black')
plt.tick_params(axis='y', colors='black')

plt.margins(x=0)
plt.show()

#### 42-qubit GSE (distance 3)

In [ ]:
mapping = 'gse3'
mapping_all_angles[mapping].append(init_angles)
mapping_updated_angles[mapping], mapping_energies[mapping] = VQE(
    mapping,
    l_rate,
    opt_iters,
    init_angles=deepcopy(init_angles),
    syndromes=mapping_syndromes[mapping],
    stat_noise_model=stat_noise_model,
    gate_noise_model=gate_noise_model,
    shots=mapping_shots[mapping],
    anglewise_opt=anglewise,
    cpus=cpus)

In [ ]:
fig = plt.figure(dpi=200)
plt.style.use('ggplot')
plt.plot(range(len(mapping_energies['noiseless'])),mapping_energies['noiseless'],label=mapping_labels['noiseless'],color = mapping_colors['noiseless'])
for mapping in ['jw','lw','gse','aqm','gse3']:
    plt.plot(range(len(mapping_energies[mapping])),mapping_energies[mapping],label=mapping_labels[mapping],color = mapping_colors[mapping])
plt.hlines(E0, 0, len(mapping_energies['noiseless']),label='$E_0$',color='black',linestyles='dashed')
plt.legend()
plt.xlabel(f'evaluations',color='black')
plt.ylabel(rf'$\langle H \rangle$',color='black')
plt.tick_params(axis='x', colors='black')
plt.tick_params(axis='y', colors='black')

plt.margins(x=0)
plt.show()
#plt.savefig(f'noisy_VQE_opt_fig.pdf',facecolor='white',format='pdf')

#### Plotting

In [ ]:
fig, axs = plt.subplots(1,2,figsize =(15,6),dpi=200)
plt.style.use('ggplot')
axs[0].plot(range(len(mapping_energies['noiseless'])),mapping_energies['noiseless'],label=mapping_labels['noiseless'],color = mapping_colors['noiseless'])
for mapping in ['jw','lw','gse','aqm','gse3']:
    axs[0].plot(range(len(mapping_energies[mapping])),mapping_energies[mapping],label=mapping_labels[mapping],color = mapping_colors[mapping])
axs[0].hlines(E0, 0, len(mapping_energies['noiseless']),label='$E_0$',color='black',linestyles='dashed')
axs[0].legend()
axs[0].set_xlabel(f'evaluations',color='black')
axs[0].set_ylabel(rf'$\langle H \rangle$',color='black')
axs[0].tick_params(axis='x', colors='black')
axs[0].tick_params(axis='y', colors='black')
axs[0].margins(x=0)

for mapping in ['jw','lw','gse','aqm','gse3']:
    axs[1].plot(range(len(mapping_energies[mapping])),np.array(mapping_energies[mapping])-np.array(mapping_energies['noiseless']),label=mapping_labels[mapping],color = mapping_colors[mapping])
axs[1].set_xlabel(f'evaluations',color='black')
axs[1].set_ylabel(r'$\langle H \rangle - \langle H \rangle_{\mathrm{Noiseless}}$',color='black')
axs[1].tick_params(axis='x', colors='black')
axs[1].tick_params(axis='y', colors='black')
axs[1].legend()
#plt.savefig(f'VQE_twopanel.pdf',facecolor='white',format='pdf')

plt.show()

#### Data saving

In [ ]:
experimental_data = {
    'init_angles': init_angles,
    'mappings': vqe_mappings,
    'updated_angles':mapping_updated_angles,
    'all_angles':mapping_all_angles,
    'energies':mapping_energies,
    'mapping_shots':mapping_shots,
    'l_rate':l_rate, 
    'stat_noise_rate': stat_noise_rate,
    'gate_noise_rate': gate_noise_rate,
    'num_shots': num_shots
}


with open(f'noisy_VQE_data_new_run.pkl', 'wb') as f:
    pickle.dump(experimental_data, f)